In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
d={
    'hash'          :'category',
    'trajectory_id' :'category',
    'time_entry'    :'category',
    'time_exit'    :'category',
    'vmax':'float64',
    'vmin':'float64',
    'vmean':'float64',
    'x_entry':'float64',
    'y_entry':'float64',
    'x_exit':'float64',
    'y_exit':'float64'
}

train = pd.read_csv('./input/data_train.csv',
                    #nrows=10,
                    dtype=d).drop(columns=['Unnamed: 0'])
test = pd.read_csv('./input/data_test.csv',
                   #nrows=10,
                   dtype=d).drop(columns=['Unnamed: 0'])

In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 814262 entries, 0 to 814261
Data columns (total 15 columns):
hash             814262 non-null category
trajectory_id    814262 non-null category
time_entry       814262 non-null category
time_exit        814262 non-null category
vmax             256769 non-null float64
vmin             256769 non-null float64
vmean            270778 non-null float64
x_entry          814262 non-null float64
y_entry          814262 non-null float64
x_exit           814262 non-null float64
y_exit           814262 non-null float64
first            814262 non-null bool
last             814262 non-null bool
x_req            814262 non-null float64
y_req            814262 non-null float64
dtypes: bool(2), category(4), float64(9)
memory usage: 128.0 MB


In [3]:
def direct(ax, ay, bx, by):
    return bx-ax, by-ay


def calc_directory(v):
    # 7,8 ->9,10 
    return direct(v[7],v[8],v[9],v[10])


def add_directories(dirx, diry, indexes, df):
    first = True
    meanx, meany = 0, 0
    for i in indexes:
        if first==True:
            first = False
            dirx[i], diry[i] = calc_directory(df.values[i])
            
        else:    
            x, y = calc_directory(df.values[last])
            meanx, meany = (x+meanx)/2, (y+meany)/2
            dirx[i], diry[i] = meanx, meany
            
        last = i
        # print(dirx[i], diry[i])
      
        
def make_directories(df):
    dirx=np.empty(len(df))
    diry=np.empty(len(df))
    drivers = df.hash.unique()
    
    for driver in drivers:
        add_directories(dirx, diry, df[df.hash == driver].index, df) 
    
    df['x_dir'] = dirx
    df['y_dir'] = diry

    


In [4]:
last=0


def new_col_first(df, col='first'):
    global last
    last=0
    def smth(row):
        global last
        if row.hash == last:
            return False
        else:
            last=row.hash
            return True    
    df[col] = df.apply(smth, axis=1)

def new_col_last(df, col='last'):
    vec=list(df['first'])
    vec.pop(0)
    vec.append(1)
    df[col]=pd.Series(vec,dtype=bool)
    
    
def new_col_requrency(df, col, col_req):
    global last
    last=0
    def smth(row,col):
        global last
        x=last
        last=row[col]
        return x    
    df[col_req] = df.apply(lambda x: smth(x,col), axis=1).astype('bool')
    
def new_col_dir(df):
    global last
    def direct(a,b):
        return b-a
    def dir_x(row):
        global last
        if row['first']==True:
            last = direct(row.x_entry,row.x_exit)
            return last
        else:
            result = last
            last = direct(row.x_entry,row.x_exit)            
            return result
    def dir_y(row):
        global last
        if row['first']==True:
            last = direct(row.y_entry,row.y_exit)   
            return last
        else:
            result = last
            last = direct(row.y_entry,row.y_exit)   
            return result    
        
    df['x_dir']= df.apply(dir_x, axis=1)  
    df['y_dir']= df.apply(dir_y, axis=1)
    
def new_col_req(df):
    global last
    def reqx(row):
        global last
        if row['first']==True:
            last = row['x_entry']
            return last
        else:
            tmp = last
            last = row['x_entry']
            return tmp
    def reqy(row):
        global last
        if row['first']==True:
            last = row['y_entry']
            return last
        else:
            tmp = last
            last = row['y_entry']
            return tmp
        
    df['x_req']= df.apply(reqx, axis=1)
    df['y_req']= df.apply(reqy, axis=1)      


In [5]:
def proceed(df):
    new_col_first(df)
    new_col_last(df)
    #new_col_dir(df)
    new_col_req(df)


proceed(train)
proceed(test)

In [14]:
def within_measure(x, y):
    #  3750901.5068 ≤ 𝑥 ≤ 3770901.5068
    #  −19268905.6133 ≤ 𝑦 ≤ −19208905.6133
    if 3750901.5068 <= x and x <= 3770901.5068 and -19268905.6133 <= y and y <= -19208905.6133:
        return 1
    else:
        return 0

def time_to_sec(values):
    return pd.to_timedelta(values).dt.total_seconds().astype(int)

train.time_entry = time_to_sec(train.time_entry)
train.time_exit = time_to_sec(train.time_exit)
test.time_entry = time_to_sec(test.time_entry)
test.time_exit = time_to_sec(test.time_exit)


In [15]:
def time(df):
    def dtime(row):
        return row.time_exit-row.time_entry
    df['time']=df.apply(dtime, axis=1)
time(train)
time(test)

In [16]:
train[:80]

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,first,last,x_req,y_req,time
0,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_0,25471,25712,NaN,NaN,NaN,3.751014e+06,-1.909398e+07,3.750326e+06,-1.913634e+07,True,False,3.751014e+06,-1.909398e+07,241
1,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_1,26434,26742,NaN,NaN,NaN,3.743937e+06,-1.932247e+07,3.744975e+06,-1.931966e+07,False,False,3.751014e+06,-1.909398e+07,308
2,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_2,28412,29005,NaN,NaN,NaN,3.744868e+06,-1.929356e+07,3.744816e+06,-1.929284e+07,False,False,3.743937e+06,-1.932247e+07,593
3,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_3,29870,31043,NaN,NaN,NaN,3.744880e+06,-1.929229e+07,3.744809e+06,-1.929049e+07,False,False,3.744868e+06,-1.929356e+07,1173
4,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_4,52689,52689,NaN,NaN,NaN,3.744909e+06,-1.928558e+07,3.744909e+06,-1.928558e+07,False,False,3.744880e+06,-1.929229e+07,0
5,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_5,54151,55113,NaN,NaN,NaN,3.744945e+06,-1.928183e+07,3.744785e+06,-1.928148e+07,False,True,3.744909e+06,-1.928558e+07,962
6,0000cf177130469eeac79f67b6bcf3df_9,traj_0000cf177130469eeac79f67b6bcf3df_9_0,52163,52163,0.000000,0.000000,0.000000,3.749450e+06,-1.926506e+07,3.749450e+06,-1.926506e+07,True,False,3.749450e+06,-1.926506e+07,0
7,0000cf177130469eeac79f67b6bcf3df_9,traj_0000cf177130469eeac79f67b6bcf3df_9_1,52789,52789,0.000000,0.000000,0.000000,3.749090e+06,-1.926726e+07,3.749090e+06,-1.926726e+07,False,False,3.749450e+06,-1.926506e+07,0
8,0000cf177130469eeac79f67b6bcf3df_9,traj_0000cf177130469eeac79f67b6bcf3df_9_2,53414,53414,0.507809,0.507809,0.507809,3.749042e+06,-1.926632e+07,3.749042e+06,-1.926632e+07,False,False,3.749090e+06,-1.926726e+07,0
9,0000cf177130469eeac79f67b6bcf3df_9,traj_0000cf177130469eeac79f67b6bcf3df_9_3,54032,55788,1.149404,1.149404,1.149404,3.749088e+06,-1.926605e+07,3.749610e+06,-1.926594e+07,False,True,3.749042e+06,-1.926632e+07,1756


In [6]:
test[:80]

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit
0,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_0,11:43:17,11:50:17,NaN,NaN,NaN,3.773413e+06,-1.909828e+07,3.773111e+06,-1.914508e+07
1,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_2,12:21:37,12:21:37,0.00,0.00,0.00,3.773199e+06,-1.914354e+07,3.773199e+06,-1.914354e+07
2,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_3,12:34:27,13:14:11,NaN,NaN,NaN,3.763760e+06,-1.921342e+07,3.771757e+06,-1.911092e+07
3,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_4,13:25:33,13:43:13,NaN,NaN,NaN,3.773385e+06,-1.911344e+07,3.773131e+06,-1.914465e+07
4,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,15:03:32,15:10:32,NaN,NaN,NaN,3.773118e+06,-1.914490e+07,NaN,NaN
5,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_0,08:08:23,08:20:08,17.29,17.29,17.29,3.771380e+06,-1.933274e+07,3.769933e+06,-1.934095e+07
6,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_6,12:08:23,12:13:24,NaN,NaN,NaN,3.771380e+06,-1.933274e+07,3.769983e+06,-1.934265e+07
7,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_10,15:29:09,15:30:56,NaN,NaN,NaN,3.769978e+06,-1.934136e+07,NaN,NaN
8,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_1,08:45:05,08:50:33,NaN,NaN,NaN,3.760880e+06,-1.910042e+07,3.759312e+06,-1.908280e+07
9,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_2,11:09:02,11:09:02,NaN,NaN,NaN,3.755349e+06,-1.916135e+07,3.755349e+06,-1.916135e+07


In [17]:
test2=test[test['last']].reset_index(drop=True)
def drop_c(df):
    df.drop(columns=['first','last','vmin','vmax'],inplace=True)
    
drop_c(train)
drop_c(test)
drop_c(test2)

trajectory_id = test2['trajectory_id']

In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 814262 entries, 0 to 814261
Data columns (total 12 columns):
hash             814262 non-null category
trajectory_id    814262 non-null category
time_entry       814262 non-null int32
time_exit        814262 non-null int32
vmean            270778 non-null float64
x_entry          814262 non-null float64
y_entry          814262 non-null float64
x_exit           814262 non-null float64
y_exit           814262 non-null float64
x_req            814262 non-null float64
y_req            814262 non-null float64
time             814262 non-null int64
dtypes: category(2), float64(7), int32(2), int64(1)
memory usage: 114.4 MB


In [19]:
train[:10]

,hash,trajectory_id,time_entry,time_exit,vmean,x_entry,y_entry,x_exit,y_exit,x_req,y_req,time
0,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_0,25471,25712,NaN,3.751014e+06,-1.909398e+07,3.750326e+06,-1.913634e+07,3.751014e+06,-1.909398e+07,241
1,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_1,26434,26742,NaN,3.743937e+06,-1.932247e+07,3.744975e+06,-1.931966e+07,3.751014e+06,-1.909398e+07,308
2,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_2,28412,29005,NaN,3.744868e+06,-1.929356e+07,3.744816e+06,-1.929284e+07,3.743937e+06,-1.932247e+07,593
3,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_3,29870,31043,NaN,3.744880e+06,-1.929229e+07,3.744809e+06,-1.929049e+07,3.744868e+06,-1.929356e+07,1173
4,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_4,52689,52689,NaN,3.744909e+06,-1.928558e+07,3.744909e+06,-1.928558e+07,3.744880e+06,-1.929229e+07,0
5,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_5,54151,55113,NaN,3.744945e+06,-1.928183e+07,3.744785e+06,-1.928148e+07,3.744909e+06,-1.928558e+07,962
6,0000cf177130469eeac79f67b6bcf3df_9,traj_0000cf177130469eeac79f67b6bcf3df_9_0,52163,52163,0.000000,3.749450e+06,-1.926506e+07,3.749450e+06,-1.926506e+07,3.749450e+06,-1.926506e+07,0
7,0000cf177130469eeac79f67b6bcf3df_9,traj_0000cf177130469eeac79f67b6bcf3df_9_1,52789,52789,0.000000,3.749090e+06,-1.926726e+07,3.749090e+06,-1.926726e+07,3.749450e+06,-1.926506e+07,0
8,0000cf177130469eeac79f67b6bcf3df_9,traj_0000cf177130469eeac79f67b6bcf3df_9_2,53414,53414,0.507809,3.749042e+06,-1.926632e+07,3.749042e+06,-1.926632e+07,3.749090e+06,-1.926726e+07,0
9,0000cf177130469eeac79f67b6bcf3df_9,traj_0000cf177130469eeac79f67b6bcf3df_9_3,54032,55788,1.149404,3.749088e+06,-1.926605e+07,3.749610e+06,-1.926594e+07,3.749042e+06,-1.926632e+07,1756


In [20]:
test[:10]

,hash,trajectory_id,time_entry,time_exit,vmean,x_entry,y_entry,x_exit,y_exit,x_req,y_req,time
0,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_0,42197,42617,NaN,3.773413e+06,-1.909828e+07,3.773111e+06,-1.914508e+07,3.773413e+06,-1.909828e+07,420
1,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_2,44497,44497,0.00,3.773199e+06,-1.914354e+07,3.773199e+06,-1.914354e+07,3.773413e+06,-1.909828e+07,0
2,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_3,45267,47651,NaN,3.763760e+06,-1.921342e+07,3.771757e+06,-1.911092e+07,3.773199e+06,-1.914354e+07,2384
3,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_4,48333,49393,NaN,3.773385e+06,-1.911344e+07,3.773131e+06,-1.914465e+07,3.763760e+06,-1.921342e+07,1060
4,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,54212,54632,NaN,3.773118e+06,-1.914490e+07,NaN,NaN,3.773385e+06,-1.911344e+07,420
5,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_0,29303,30008,17.29,3.771380e+06,-1.933274e+07,3.769933e+06,-1.934095e+07,3.771380e+06,-1.933274e+07,705
6,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_6,43703,44004,NaN,3.771380e+06,-1.933274e+07,3.769983e+06,-1.934265e+07,3.771380e+06,-1.933274e+07,301
7,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_10,55749,55856,NaN,3.769978e+06,-1.934136e+07,NaN,NaN,3.771380e+06,-1.933274e+07,107
8,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_1,31505,31833,NaN,3.760880e+06,-1.910042e+07,3.759312e+06,-1.908280e+07,3.760880e+06,-1.910042e+07,328
9,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_2,40142,40142,NaN,3.755349e+06,-1.916135e+07,3.755349e+06,-1.916135e+07,3.760880e+06,-1.910042e+07,0


In [21]:
test2[:10]

,hash,trajectory_id,time_entry,time_exit,vmean,x_entry,y_entry,x_exit,y_exit,x_req,y_req,time
0,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,54212,54632,NaN,3.773118e+06,-1.914490e+07,NaN,NaN,3.773385e+06,-1.911344e+07,420
1,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_10,55749,55856,NaN,3.769978e+06,-1.934136e+07,NaN,NaN,3.771380e+06,-1.933274e+07,107
2,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_5,55568,55568,NaN,3.757468e+06,-1.923860e+07,NaN,NaN,3.755349e+06,-1.916135e+07,0
3,0005401ceddaf27a9b7f0d42ef1fbe95_1,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,56118,56525,NaN,3.760505e+06,-1.935500e+07,NaN,NaN,3.751349e+06,-1.916284e+07,407
4,00063a4f6c12e1e4de7d876580620667_3,traj_00063a4f6c12e1e4de7d876580620667_3_4,53647,54314,NaN,3.766319e+06,-1.917013e+07,NaN,NaN,3.776264e+06,-1.918289e+07,667
5,0006535be25bb52dd06983447880c964_5,traj_0006535be25bb52dd06983447880c964_5_12,55326,55326,NaN,3.776037e+06,-1.916993e+07,NaN,NaN,3.769801e+06,-1.935089e+07,0
6,0006f84bb33ec929d1cda7686f861d0a_31,traj_0006f84bb33ec929d1cda7686f861d0a_31_3,54012,54012,NaN,3.760009e+06,-1.923361e+07,NaN,NaN,3.759835e+06,-1.924379e+07,0
7,00093ae562586aed0e053b8431e8ace4_23,traj_00093ae562586aed0e053b8431e8ace4_23_10,57189,57189,NaN,3.756665e+06,-1.914692e+07,NaN,NaN,3.756680e+06,-1.914717e+07,0
8,000c739e444a70e1804d757a0580caaa_31,traj_000c739e444a70e1804d757a0580caaa_31_3,56491,56491,NaN,3.768082e+06,-1.922134e+07,NaN,NaN,3.768081e+06,-1.922126e+07,0
9,000d479078af08618bddc7f09082b8c3_11,traj_000d479078af08618bddc7f09082b8c3_11_6,53886,54220,NaN,3.766505e+06,-1.920176e+07,NaN,NaN,3.744812e+06,-1.932195e+07,334


In [22]:

for col in ['time_entry','time_exit']:
    mode = train[col].mode()[0]
    test[col].fillna(mode, inplace=True)
    test2[col].fillna(mode, inplace=True)
    train[col].fillna(mode, inplace=True)
    
for col in ['vmean']:
    mode = train[col].mode()[0]
    test[col].fillna(0, inplace=True)
    test2[col].fillna(0, inplace=True)
    train[col].fillna(0, inplace=True)    

In [23]:
train[:20]

,hash,trajectory_id,time_entry,time_exit,vmean,x_entry,y_entry,x_exit,y_exit,x_req,y_req,time
0,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_0,25471,25712,0.000000,3.751014e+06,-1.909398e+07,3.750326e+06,-1.913634e+07,3.751014e+06,-1.909398e+07,241
1,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_1,26434,26742,0.000000,3.743937e+06,-1.932247e+07,3.744975e+06,-1.931966e+07,3.751014e+06,-1.909398e+07,308
2,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_2,28412,29005,0.000000,3.744868e+06,-1.929356e+07,3.744816e+06,-1.929284e+07,3.743937e+06,-1.932247e+07,593
3,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_3,29870,31043,0.000000,3.744880e+06,-1.929229e+07,3.744809e+06,-1.929049e+07,3.744868e+06,-1.929356e+07,1173
4,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_4,52689,52689,0.000000,3.744909e+06,-1.928558e+07,3.744909e+06,-1.928558e+07,3.744880e+06,-1.929229e+07,0
5,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_5,54151,55113,0.000000,3.744945e+06,-1.928183e+07,3.744785e+06,-1.928148e+07,3.744909e+06,-1.928558e+07,962
6,0000cf177130469eeac79f67b6bcf3df_9,traj_0000cf177130469eeac79f67b6bcf3df_9_0,52163,52163,0.000000,3.749450e+06,-1.926506e+07,3.749450e+06,-1.926506e+07,3.749450e+06,-1.926506e+07,0
7,0000cf177130469eeac79f67b6bcf3df_9,traj_0000cf177130469eeac79f67b6bcf3df_9_1,52789,52789,0.000000,3.749090e+06,-1.926726e+07,3.749090e+06,-1.926726e+07,3.749450e+06,-1.926506e+07,0
8,0000cf177130469eeac79f67b6bcf3df_9,traj_0000cf177130469eeac79f67b6bcf3df_9_2,53414,53414,0.507809,3.749042e+06,-1.926632e+07,3.749042e+06,-1.926632e+07,3.749090e+06,-1.926726e+07,0
9,0000cf177130469eeac79f67b6bcf3df_9,traj_0000cf177130469eeac79f67b6bcf3df_9_3,54032,55788,1.149404,3.749088e+06,-1.926605e+07,3.749610e+06,-1.926594e+07,3.749042e+06,-1.926632e+07,1756


In [24]:
test2[:20]

,hash,trajectory_id,time_entry,time_exit,vmean,x_entry,y_entry,x_exit,y_exit,x_req,y_req,time
0,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,54212,54632,0.0,3.773118e+06,-1.914490e+07,NaN,NaN,3.773385e+06,-1.911344e+07,420
1,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_10,55749,55856,0.0,3.769978e+06,-1.934136e+07,NaN,NaN,3.771380e+06,-1.933274e+07,107
2,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_5,55568,55568,0.0,3.757468e+06,-1.923860e+07,NaN,NaN,3.755349e+06,-1.916135e+07,0
3,0005401ceddaf27a9b7f0d42ef1fbe95_1,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,56118,56525,0.0,3.760505e+06,-1.935500e+07,NaN,NaN,3.751349e+06,-1.916284e+07,407
4,00063a4f6c12e1e4de7d876580620667_3,traj_00063a4f6c12e1e4de7d876580620667_3_4,53647,54314,0.0,3.766319e+06,-1.917013e+07,NaN,NaN,3.776264e+06,-1.918289e+07,667
5,0006535be25bb52dd06983447880c964_5,traj_0006535be25bb52dd06983447880c964_5_12,55326,55326,0.0,3.776037e+06,-1.916993e+07,NaN,NaN,3.769801e+06,-1.935089e+07,0
6,0006f84bb33ec929d1cda7686f861d0a_31,traj_0006f84bb33ec929d1cda7686f861d0a_31_3,54012,54012,0.0,3.760009e+06,-1.923361e+07,NaN,NaN,3.759835e+06,-1.924379e+07,0
7,00093ae562586aed0e053b8431e8ace4_23,traj_00093ae562586aed0e053b8431e8ace4_23_10,57189,57189,0.0,3.756665e+06,-1.914692e+07,NaN,NaN,3.756680e+06,-1.914717e+07,0
8,000c739e444a70e1804d757a0580caaa_31,traj_000c739e444a70e1804d757a0580caaa_31_3,56491,56491,0.0,3.768082e+06,-1.922134e+07,NaN,NaN,3.768081e+06,-1.922126e+07,0
9,000d479078af08618bddc7f09082b8c3_11,traj_000d479078af08618bddc7f09082b8c3_11_6,53886,54220,0.0,3.766505e+06,-1.920176e+07,NaN,NaN,3.744812e+06,-1.932195e+07,334


In [25]:
train.to_csv('./output/train.csv',index=False)    
test.to_csv('./output/test.csv',index=False)    
test2.to_csv('./output/test2.csv',index=False)    